In [12]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [13]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [36]:
epochs = 20
batches = 600

In [37]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=[None, 28, 28, 1], name = "X")
Y = tf.placeholder(tf.float32, shape=[None, 10], name = "Y")

conv1 = tf.layers.conv2d(X, filters=32, kernel_size=[5, 5], strides=[1, 1], padding='same', name = "conv1")
pool1 = tf.layers.max_pooling2d(conv1, pool_size=[2, 2], strides=[2, 2], padding='same', name = "pool1")

conv2 = tf.layers.conv2d(pool1, filters=64, kernel_size=[5, 5], strides=[1, 1], padding='same', name = "conv2")
pool2 = tf.layers.max_pooling2d(conv2, pool_size=[2, 2], strides=[2, 2], padding='same', name = "pool2")
    
# Logits
logits = tf.layers.dense(tf.layers.flatten(pool2), units=10)

# Predictor
f = tf.nn.softmax(logits)

# Cross Entropy loss
loss = tf.losses.softmax_cross_entropy(Y, logits)
trainer = tf.train.AdamOptimizer().minimize(loss)
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(f, 1), tf.argmax(Y, 1)), tf.float32))

saver = tf.train.Saver()
init = tf.global_variables_initializer()


In [39]:
with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess,"Models/MNIST_numbers.ckpt")
    for i in range(epochs):
        
        data, label = mnist.train.next_batch(batches)
        data = data.reshape(batches, 28, 28, 1)
        
        feed = {X: data, Y: label}
        sess.run(trainer, feed_dict = feed)
    
    saver.save(sess,"Models/MNIST_numbers.ckpt")
    
    x_train = np.reshape(mnist.test.images, (-1, 28, 28, 1))  # tf.layers.conv2d takes 4D input
    
    acc = sess.run(accuracy,feed_dict={X: x_train, Y: mnist.test.labels})
    
    print("acc: "+ str(acc))
    sess.close()

acc: 0.8636
